# List of all chat models

In [4]:
import requests, json

r = requests.get("https://openrouter.ai/api/v1/models", timeout=30)
r.raise_for_status()
model_info = r.json()['data']


In [9]:
import pandas as pd 
pd.DataFrame(model_info)

,id,canonical_slug,hugging_face_id,name,created,description,context_length,architecture,pricing,top_provider,per_request_limits,supported_parameters
0,deepcogito/cogito-v2-preview-llama-109b-moe,deepcogito/cogito-v2-preview-llama-109b-moe,deepcogito/cogito-v2-preview-llama-109B-MoE,Cogito V2 Preview Llama 109B,1756831568,"An instruction-tuned, hybrid-reasoning Mixture...",32767,"{'modality': 'text+image->text', 'input_modali...","{'prompt': '0.00000018', 'completion': '0.0000...","{'context_length': 32767, 'max_completion_toke...",None,"[frequency_penalty, include_reasoning, logit_b..."
1,deepcogito/cogito-v2-preview-deepseek-671b,deepcogito/cogito-v2-preview-deepseek-671b,deepcogito/cogito-v2-preview-deepseek-671B-MoE,Deep Cogito: Cogito V2 Preview Deepseek 671B,1756830949,"Cogito v2 is a multilingual, instruction-tuned...",163840,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.00000125', 'completion': '0.0000...","{'context_length': 163840, 'max_completion_tok...",None,"[frequency_penalty, include_reasoning, logit_b..."
2,qwen/qwen3-30b-a3b-thinking-2507,qwen/qwen3-30b-a3b-thinking-2507,Qwen/Qwen3-30B-A3B-Thinking-2507,Qwen: Qwen3 30B A3B Thinking 2507,1756399192,Qwen3-30B-A3B-Thinking-2507 is a 30B parameter...,262144,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.0000000713', 'completion': '0.00...","{'context_length': 262144, 'max_completion_tok...",None,"[frequency_penalty, include_reasoning, logit_b..."
3,x-ai/grok-code-fast-1,x-ai/grok-code-fast-1,,xAI: Grok Code Fast 1,1756238927,Grok Code Fast 1 is a speedy and economical re...,256000,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.0000002', 'completion': '0.00000...","{'context_length': 256000, 'max_completion_tok...",None,"[include_reasoning, logprobs, max_tokens, reas..."
4,nousresearch/hermes-4-70b,nousresearch/hermes-4-70b,NousResearch/Hermes-4-70B,Nous: Hermes 4 70B,1756236182,Hermes 4 70B is a hybrid reasoning model from ...,131072,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.00000009329544', 'completion': '...","{'context_length': 131072, 'max_completion_tok...",None,"[frequency_penalty, include_reasoning, logit_b..."
...,...,...,...,...,...,...,...,...,...,...,...,...
317,undi95/remm-slerp-l2-13b,undi95/remm-slerp-l2-13b,Undi95/ReMM-SLERP-L2-13B,ReMM SLERP 13B,1689984000,A recreation trial of the original MythoMax-L2...,6144,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.00000045', 'completion': '0.0000...","{'context_length': 6144, 'max_completion_token...",None,"[frequency_penalty, logit_bias, max_tokens, mi..."
318,gryphe/mythomax-l2-13b,gryphe/mythomax-l2-13b,Gryphe/MythoMax-L2-13b,MythoMax 13B,1688256000,One of the highest performing and most popular...,4096,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.00000006', 'completion': '0.0000...","{'context_length': 4096, 'max_completion_token...",None,"[frequency_penalty, logit_bias, max_tokens, mi..."
319,openai/gpt-4-0314,openai/gpt-4-0314,None,OpenAI: GPT-4 (older v0314),1685232000,GPT-4-0314 is the first version of GPT-4 relea...,8191,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.00003', 'completion': '0.00006',...","{'context_length': 8191, 'max_completion_token...",None,"[frequency_penalty, logit_bias, logprobs, max_..."
320,openai/gpt-3.5-turbo,openai/gpt-3.5-turbo,None,OpenAI: GPT-3.5 Turbo,1685232000,GPT-3.5 Turbo is OpenAI's fastest model. It ca...,16385,"{'modality': 'text->text', 'input_modalities':...","{'prompt': '0.0000005', 'completion': '0.00000...","{'context_length': 16385, 'max_completion_toke...",None,"[frequency_penalty, logit_bias, logprobs, max_..."


# Benchmark data

In [28]:
"""
Download current benchmark data for LLMs from:
  1) LiveBench leaderboard judgments (Parquet)
  2) Our World in Data AI capabilities test scores (CSV/JSON)

Uses graze utilities for robust URL -> file downloads.
Encapsulates logic in two functions: acquire_livebench_data and acquire_owid_data.
"""

import os
import json
import time
import pathlib
from pathlib import Path
from typing import Optional, Tuple, List
from functools import partial
import io
import zipfile
import requests

from aix.util import model_info_dir

# graze helpers
from graze.base import url_to_file_download, return_filepath

# --------- Config ---------
LIVEBENCH_PARQUET_URL = (
    "https://huggingface.co/datasets/livebench/model_judgment/"
    "resolve/main/data/leaderboard-00000-of-00001.parquet"
)

# Global: whether to force-refresh downloads even if files exist
REFRESH = False


def acquire_livebench_data(model_dir: Path, *, refresh: Optional[bool] = None) -> Optional[Path]:
    if refresh is None:
        refresh = REFRESH

    lb_path = model_dir / "livebench_model_judgment.parquet"

    if lb_path.exists() and not refresh:
        return lb_path

    print(f"Acquiring LiveBench to: {lb_path}")

    try:
        ret = url_to_file_download(
            LIVEBENCH_PARQUET_URL,
            filepath=str(lb_path),
            overwrite=bool(refresh),
            return_func=return_filepath,
        )
        if not ret:
            print("LiveBench: download helper returned nothing")
            return None
        lb_file = Path(ret)
        if lb_file.exists():
            print(f"✓ LiveBench: {lb_file.name} ({lb_file.stat().st_size:,} bytes)")
            return lb_file
        else:
            print("LiveBench: file does not exist after download attempt")
            return None
    except Exception as e:
        print(f"!!! LiveBench download failed: {e}")
        return None


# ---- Run acquisition ----
model_info_path_obj = Path(model_info_dir)

lb_file = acquire_livebench_data(model_info_path_obj)



Acquiring OWID CSV for 'ai_vs_humans' -> https://ourworldindata.org/grapher/test-scores-ai-capabilities-relative-human-performance.csv?useColumnShortNames=true
OWID CSV for 'ai_vs_humans' is non-redistributable; not available for download.

Done.
Saved files:
  • /Users/thorwhalen/.config/aix/model_info/livebench_model_judgment.parquet
  • /Users/thorwhalen/.config/aix/model_info/owid_ai_vs_humans.chart.json
OWID CSV for 'ai_vs_humans' is non-redistributable; not available for download.

Done.
Saved files:
  • /Users/thorwhalen/.config/aix/model_info/livebench_model_judgment.parquet
  • /Users/thorwhalen/.config/aix/model_info/owid_ai_vs_humans.chart.json


In [ ]:
import pyckup